In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics

1.  의사결정나무의 수동구현은 위에서 제시된 모듈 (numpy, pandas,
    sklearn.metrics, matplotlib, seaborn) 만을 사용해야하며 이외의
    모듈을 사용할 경우 0점 처리함.
2.  True/False를 판단하는 문제는 답만 써도 무방함. (이유를 써도
    상관없음)
3.  Treu/False의 판단 문제는 모두 맞출 경우만 정답으로 인정함. 다만
    틀린이유가 사소하다고 판단할경우 감점없이 만점처리함.

# 1. 의사결정나무의 수동구현 (70점)

In [165]:
df_train = pd.read_csv('https://raw.githubusercontent.com/guebin/MP2023/master/posts/height_train.csv')
df_train

,weight,sex,height
0,NaN,male,164.227738
1,NaN,male,165.798660
2,75.219015,male,165.528672
3,NaN,male,163.706442
4,81.476750,male,165.501403
...,...,...,...
275,49.308558,female,148.587771
276,NaN,male,164.822474
277,NaN,male,163.907671
278,NaN,male,161.674476


`(1)` `df_train`에서 “sex”,“weight”를 설명변수로 “height”을 반응변수로
설정하라. 결측치가 있을 경우 결측값에 일괄적으로 -99로 채워넣어라.

In [166]:
df_train.isnull().sum() # weihgt 129개의 missing value.

weight    129
sex         0
height      0
dtype: int64

In [167]:
df_train.fillna(-99, inplace=True)
df_train.isnull().sum()

weight    0
sex       0
height    0
dtype: int64

In [168]:
X = df_train[['sex','weight']]
y = df_train[['height']]

`(2)` `height`열의 평균으로 `height`의 값을 추정하라. 추정값을 `yhat`에
저장하라. `sklearn.metrics.r2_score()`을 이용하여 `r2_score`를 계산하라.

**hint:** 0이 나와야 한다.

In [46]:
tb1 = pd.DataFrame({'y':df_train['height'], 'yhat':pd.Series([df_train['height'].mean()]*len(df_train))})
tb1.head()

,y,yhat
0,164.227738,157.986528
1,165.798660,157.986528
2,165.528672,157.986528
3,163.706442,157.986528
4,165.501403,157.986528


In [48]:
r2_score = sklearn.metrics.r2_score(tb1['y'], tb1['yhat'])
r2_score

0.0

`(3)` 아래를 계산하라.

-   `r` = `y` - `yhat`

여기에서 `yhat`은 `(2)`의 결과로 얻어진 적합값을 의미한다. 이제 `r`에
Weight를 기준으로 의사결정나무를 적용하여 아래와 같은 분할을 만들어라.

-   `X['weight']` \< `c`
-   `X['weight']` \>= `c`

`sklearn.metrics.r2_score()`를 이용하여 최적의 $c$값을 구하여라.

**참고**

아래의 구간을 적당히 등분하여 구할 것. 너무 세밀하게 등분하지 않아도
무방함.

``` python
(X['weight'].min(), X['weight'].max())
```

In [79]:
yhat = tb1['yhat']
r = tb1['y'] - tb1['yhat']

In [158]:
weight_range = (X['weight'].min(), X['weight'].max())
# weight_range = (X[X['weight']>0]['weight'].min(), X['weight'].max())

In [159]:
best_c = None
best_r2_score = 0

for c in range(int(weight_range[0]), int(weight_range[1]) + 1):
    df_train['weight_split'] = df_train['weight'] < c
    y_pred = df_train['weight_split'].map({True: df_train.loc[df_train['weight_split'], 'height'].mean(),
                                           False: df_train.loc[~df_train['weight_split'], 'height'].mean()})
    current_r2_score = sklearn.metrics.r2_score(df_train['height'], y_pred)

    if current_r2_score > best_r2_score:
        best_r2_score = current_r2_score
        best_c = c

print("최적의 c:", best_c)
print("최적의 R-squared:", best_r2_score)

최적의 c: -98
최적의 R-squared: 0.5320524394291483


`(4)` `(3)`의 결과로 얻어진 아래의 분할을 고려하자.

``` python
X['weight'] >= c
```

이 분할에서 depth=2 로 나무를 성장하고자 한다. 성장이가능한가? 성장이
가능하다면 이때 나무를 성장시키기 위한 변수로 weigth와 sex중 무엇이
적절한가? 왜 그렇다고 생각하는가?

- 성장가능하다. sex가 더 적절하다고 생각한다. sex를 기준으로 분기하게 되면 해당 노드에서 데이터를 더 효과적으로 구분할 수 있기 때문이다.

In [162]:
df_train.groupby('sex').agg({'height':'mean'})

,height
sex,
female,147.436329
male,164.915949


In [157]:
y_pred_depth1 = (df_train['weight'] >= -98).astype(int)
r_fit_depth1 = df_train['height'] - y_pred_depth1

# depth=1
df_train['weight_split'] = df_train['weight'] < -98
mean_left_depth1 = df_train.loc[df_train['weight_split'], 'height'].mean()
mean_right_depth1 = df_train.loc[~df_train['weight_split'], 'height'].mean()
y_pred_depth1 = df_train['weight_split'].map({True: mean_left_depth1, False: mean_right_depth1})

# R-squared 계산
r2_score_depth1 = sklearn.metrics.r2_score(df_train['height'], y_pred_depth1)

# depth=1
print("Depth=1에서의 분할 변수: weight")
print("Depth=1에서의 R-squared:", r2_score_depth1)

# r의 적합값 계산
r_fit_depth2 = df_train['height'] - y_pred_depth1

# depth=2
df_train['sex_split'] = df_train['sex'] 
mean_left_depth2 = df_train.loc[df_train['sex_split'] == 0, 'height'].mean()
mean_right_depth2 = df_train.loc[df_train['sex_split'] == 1, 'height'].mean()
y_pred_depth2 = df_train['sex_split'].map({0: mean_left_depth2, 1: mean_right_depth2})

# R-squared
r2_score_depth2 = sklearn.metrics.r2_score(df_train['height'], yhat + 0.5 * r_fit_depth2)

print("\nDepth=2에서의 분할 변수: sex")
print("Depth=2에서의 R-squared:", r2_score_depth2)

Depth=1에서의 분할 변수: weight
Depth=1에서의 R-squared: 0.5320524394291483

Depth=2에서의 분할 변수: sex
Depth=2에서의 R-squared: 0.3509606704281393


`(5)` `(3)`의 결과로 얻어진 아래의 분할을 고려하자.

``` python
X['weight'] < c
```

이 분할에서 depth=2 로 나무를 성장하고자 한다. 성장이가능한가? 성장이
가능하다면 이때 나무를 성장시키기 위한 변수로 weigth와 sex중 무엇이
적절한가? 왜 그렇다고 생각하는가?

- 성장이 가능하지 않다.

`(6)` `(3)`-`(5)`의 결과를 이용하여 `depth=2`인 의사결정나무에 의한
`r`의 적합값을 구하여라. 이를 이용하여 `yhat`을 update하라. 이때
학습률은 0.1로 설정하고 업데이트된 결과를 `yhat2`로 저장하라.
그리고`sklearn.metrics.r2_score()`을 이용하여 `y`와 `yhat2`의
`r2_score`를 계산하라.

힌트: 아래의 알고리즘이 동치임을 이용하라.

-   `yhat2` $\leftarrow$ `yhat` + 학습률 $\times$ `rhat`
-   `r2` $\leftarrow$ `r` - 학습률 $\times$ `rhat`, where `r2` = `y` -
    `yhat2`

In [147]:
# R-squared 계산
r2_score_depth2 = sklearn.metrics.r2_score(df_train['height'], yhat + 0.1* r_fit_depth2)

# depth=2에서의 결과 출력
print("\nDepth=2에서의 분할 변수: sex")
print("Depth=2에서의 R-squared:", r2_score_depth2)


Depth=2에서의 분할 변수: sex
Depth=2에서의 R-squared: 0.08891003650846196


`(7)` `(6)`에서 학습률이 0.5일 경우 `y`와 `yhat2`의 `r2_score`를
계산하라.

In [148]:
# R-squared 계산
r2_score_depth2 = sklearn.metrics.r2_score(df_train['height'], yhat + 0.5* r_fit_depth2)

# depth=2에서의 결과 출력
print("\nDepth=2에서의 분할 변수: sex")
print("Depth=2에서의 R-squared:", r2_score_depth2)


Depth=2에서의 분할 변수: sex
Depth=2에서의 R-squared: 0.3509606704281393


# 2. 다음을 읽고 참거짓을 판단하라. (30점)

`(1)` 의사결정나무에서 `max_depth`의 설정값이 커질수록 오버피팅의 위험이
있다. **(True)**

`(2)` 배깅의 설명변수중 일부를 drop 하며 나무를 성장시킨다. **(False)**

`(3)` 랜덤포레스트는 나무가지를 랜덤으로 성장시키기도 하고 파괴시키기도
한다. **(True)**

`(4)` 부스팅은 여러가지 의사결정나무의 적합값을 평균내는 방식으로
최종예측을 한다. **(False)**

`(5)` `Accuracy`는 분류문제에서 언제나 가장 합리적인 평가지표이다. **(False)**

`(6)` 모듈49의 아래그림은 “sex == ‘male’” 일 경우 “sex=‘female’”
일때보다 항상(=모든 관측치에 대하여) 키의 예측값을 +2.1 만큼 보정해야
한다는 것을 의미한다. **(False)**

![](https://guebin.github.io/MP2023/posts/13wk-49_files/figure-html/cell-14-output-2.png)

`(7)` 모듈54에서 제시된 아래의 그림은 사실 전혀 고려할 필요가 없다.

![](https://guebin.github.io/MP2023/posts/13wk-54_files/figure-html/cell-6-output-6.png)

왜냐하면 Exercise는 범주형, Weight_Loss는 연속형이므로 correlation값은
의미가 없기 때문이다. **(False)**

`(8)` 시계열분석에서 `static_feature`란 이미 알고있는 미래의
시계열자료를 의미한다. **(False)**

`(9)` 모듈59에서 소개된 자전거대여자료와 같이 시간특징이 포함된 자료는
언제나 (과거를 기반으로 미래를 예측하는) 시계열분석을 하는 것이
올바르다. **(False)**

`(10)` 모듈60에서 소개된 하이퍼파라메터 설정법을 이용하면 때때로 모형의
적합도를 높일 수 있다. **(True)**